<a href="https://colab.research.google.com/github/MicroprocessorX069/Epipolar-Geometry-/blob/master/Epipolar_Geometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

In [0]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt

def findKeypoints(input_name,output_name):
  img = cv.imread(input_name)
  gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  sift = cv.xfeatures2d.SIFT_create()
  kp, des = sift.detectAndCompute(gray,None)
  img=cv.drawKeypoints(gray,kp,img)
  #cv.imshow("Image", img)
  #cv.waitKey(0)
  #cv.destroyAllWindows()
  cv.imwrite(output_name,img)


def knnMatching(input_name1,input_name2,output_name):
  img1 = cv.imread(input_name1,0)          # queryImage
  img2 = cv.imread(input_name2,0) # trainImage
  # Initiate SIFT detector
  sift = cv.xfeatures2d.SIFT_create()
  # find the keypoints and descriptors with SIFT
  kp1, des1 = sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  # BFMatcher with default params
  bf = cv.BFMatcher()
  matches = bf.knnMatch(des1,des2, k=2)
  # Apply ratio test
  good = []
  for m,n in matches:
      if m.distance < 0.75*n.distance:
          good.append([m])
  img3=np.array([])       
  # cv.drawMatchesKnn expects list of lists as matches.
  img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,good,outImg=img3,flags=2)
  cv.imwrite(output_name,img3)
 
def featureMatching(input_name1,input_name2,output_name):
  Min_matches=10
  img1=cv.imread(input_name1,0)
  img2=cv.imread(input_name2,0)

  sift=cv.xfeatures2d.SIFT_create()
  kp1, des1=sift.detectAndCompute(img1, None) # Here mask is None.
  kp2, des2=sift.detectAndCompute(img2, None)

  Flann_index=1
  #dict() is just a function to create a dictionary of arguments 
  #More arguments can be kept than required by the method

  index_param= dict(algorithm=Flann_index,trees=5)
  search_param=dict(checks=50)
  flann=cv.FlannBasedMatcher(index_param,search_param)
  matches=flann.knnMatch(des1,des2,k=2)
  good_matches=[]
  for m,n in matches:
    if m.distance<0.7*n.distance:
      good_matches.append(m)

  #Set a condition of n matches to find an object
  #If found, extract the locations, to find the perspective transform

  if(len(good_matches)>Min_matches):
    #queryIdx and trainIdx are the descriptors of the good matched locations
    src_pts=np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    dst_pts=np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    M,mask=cv.findHomography(src_pts,dst_pts,cv.RANSAC,5.0)
    matchesMask=mask.ravel().tolist()

    h,w=img1.shape
    pts=np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
    dst=cv.perspectiveTransform(pts,M)

    img2=cv.polylines(img2,[np.int32(dst)],True,255,3,cv.LINE_AA)

  else:
    print("not enoug matches")
    matchesMask=None

  draw_param=dict(matchColor=(0,255,0), singlePointColor=None, matchesMask= matchesMask, flags=2)
  imgQ4= cv.drawMatches(img1,kp1,img2,kp2,good_matches,None,**draw_param)
  cv.imwrite(output_name,imgQ4)

In [4]:
#2.1
findKeypoints("tsucuba_left.png","task2_sift1.jpg")
findKeypoints("tsucuba_right.png","task2_sift2.jpg")
knnMatching("tsucuba_left.png","tsucuba_right.png","task2_matches_knn.jpg")
#2.2
calculateFundamentalMatrix("tsucuba_left.png","tsucuba_right.png")
#2.3
plotEpilines("tsucuba_left.png","tsucuba_right.png","task2_epi_right.jpg", "task2_epi_.left.jpg")
#2.4
disparityMap("tsucuba_left.png","tsucuba_right.png","task2 disparity.jpg")

AttributeError: ignored

In [0]:
#2 Functions
#2.2
import cv2 as cv
import numpy as np

def calculateFundamentalMatrix(input_name1,input_name2):
  img1=cv.imread(input_name1,0)
  img2=cv.imread(input_name2,0)
  sift=cv.xfeatures2d.SIFT_create()
  kp1, des1= sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  bf=cv.BFMatcher()
  matches=bf.knnMatch(des1,des2,k=2)
  good_matches=[]#?
  pts1=[]#?
  pts2=[] #?
  for m,n in matches:
    #print(m," ",n)
    if m.distance<0.75*n.distance: #?
      good_matches.append([m])
      pts1.append(kp1[m.queryIdx].pt) #.queryIdx
      pts2.append(kp2[m.trainIdx].pt)
  pts1 = np.int32(pts1)
  pts2 = np.int32(pts2)
  Fundamental_mat, mask= cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
  #print(Fundamental_mat)

 #2.3
def drawlines(input_image1,input_image2, lines,pts1, pts2):
  img1=cv.imread(input_image1,0)
  img2=cv.imread(input_image2,0)
  h,w=img1.shape
  
  img1=cv.cvtColor(img1,cv.COLOR_GRAY2BGR)
  img2=cv.cvtColor(img2,cv.COLOR_GRAY2BGR)
  for h,pt1,pt2 in zip(lines,pts1,pts2):
    
    color=tuple(np.random.randint(0,255,3).tolist())
    x0, y0= map(int, [0, -h[2]/h[1]])
    x1, y1= map(int, [w,-(h[2]+h[0]*w)/h[1]])
    img1 = cv.line(img1, (x0,y0), (x1,y1), color,1)
    img1 = cv.circle(img1,tuple(pt1),5,color,-1)
    img2 = cv.circle(img2,tuple(pt2),5,color,-1)
  return img1,img2

def plotEpilines(input_name1,input_name2,output_name1, output_name2):
  img1=cv.imread(input_name1,0)
  img2=cv.imread(input_name2,0)
  sift=cv.xfeatures2d.SIFT_create()
  kp1, des1= sift.detectAndCompute(img1, None)
  kp2, des2=sift.detectAndCompute(img2, None)
  
  bf=cv.BFMatcher()
  matches=bf.knnMatch(des1, des2, k=2)
  good_matches=[]
  pts1=[]
  pts2=[]
  for m,n in matches:
    if m.distance < 0.75*n.distance:
      good_matches.append([m])
      pts1.append(kp1[m.queryIdx].pt)
      pts2.append(kp2[m.trainIdx].pt)
  pts1 = np.int32(pts1)
  pts2 = np.int32(pts2)
  F, mask= cv.findFundamentalMat(pts1,pts2,cv.FM_LMEDS)
  pts1 = pts1[mask.ravel()==1]
  pts2 = pts2[mask.ravel()==1]
  # Find epilines corresponding to points in right image (second image) and
  # drawing its lines on left image
  lines1 = cv.computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
  lines1 = lines1.reshape(-1,3)
  img5,img6 = drawlines(input_name1,input_name2,lines1,pts1,pts2)
  # Find epilines corresponding to points in left image (first image) and
  # drawing its lines on right image
  lines2 = cv.computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
  lines2 = lines2.reshape(-1,3)
  img3,img4 = drawlines(input_name1,input_name2,lines2,pts2,pts1)
  cv.imwrite(output_name1,img3)
  cv.imwrite(output_name2,img4)
  
#2.4
def disparityMap(input_name1,input_name2,output_name):
  imgL = cv.imread(input_name1,0)
  imgR = cv.imread(input_name2,0)
  stereo = cv.createStereoBM(numDisparities=16, blockSize=15)
  disparity = stereo.compute(imgL,imgR)
  cv.imwrite(output_name,disparity)



In [0]:
#Q1.5.
imgQ42 = cv.warpPerspective(img1, M, (img2.shape[1],img2.shape[0]))
cv.imwrite("task1 pano.jpg",imgQ42)

True